In [1]:
import time
import os
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from siriuspy.search import IDSearch
from siriuspy.devices import IVU, SOFB, HLFOFB, IDFF, Tune, DVFImgProc

import epics
from mathphys.functions import save, load

timeout = 3

# Functions definition

In [2]:
def initialize_ivu(beamline):
    # Search ID
    devnameivu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    ivu = IVU(devname=devnameivu)

    # Disable beamline control
    ivu.cmd_beamline_ctrl_disable()
    print('beamline control: ', ivu.is_beamline_ctrl_enabled)

    # Disable center mode and pitch mode
    ivu.set_center_mode(False)
    ivu.set_pitch_mode(False)
    time.sleep(0.5)
    print('center mode: ', ivu.center_mode_status)
    print('pitch mode: ', ivu.pitch_mode_status)

    # Set gap speed
    ivu.set_gap_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(ivu.gap_speed))

    # Set gap to parked condition
    ivu.set_gap(ivu.kparameter_parked)
    time.sleep(0.5)
    print('gap: {:.3f} mm'.format(ivu.gap))

    return ivu

def move_ivu_gap(ivu:IVU, gap, timeout, verbose=False):
    ivu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(ivu.gap)) if verbose else 0
    if ivu.cmd_move_gap_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while ivu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(ivu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(ivu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def move_ivu_gap_robust(ivu:IVU, gap, timeout, maxiter=3, verbose=False):
    sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    i=0
    while not sucess and i<maxiter:
        i += 1
        print('Trying {:0f}/{:.0f}'.format(i, maxiter)) if verbose else 0
        ivu.cmd_reset(timeout=timeout)
        time.sleep(0.5)
        sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    if sucess:
        print('Movimentation done!\n')
        return True
    else:
        print('Error while moving.\n')
        return False

def get_data(inj_eff, lifetime, tune, nr_acq=10, acq_sleep=0.5):
    tunex_data = np.zeros(nr_acq)
    tuney_data = np.zeros(nr_acq)
    lifetime_data = np.zeros(nr_acq)
    inj_eff_data = np.zeros(nr_acq)
    for i in range(nr_acq):
        tunex_data[i] = tune.tunex
        tuney_data[i] = tune.tuney
        lifetime_data[i] = lifetime.get()
        inj_eff_data[i] = inj_eff.get()
        time.sleep(acq_sleep)
    return tunex_data, tuney_data, inj_eff_data, lifetime_data


## Search devnames

In [3]:
devnameivu_ema = IDSearch.conv_beamline_2_idname(beamline='EMA')
devnameivu_paineira = IDSearch.conv_beamline_2_idname(beamline='PAINEIRA')

## Initialize devices

In [12]:
ivu = initialize_ivu(beamline='EMA')
ivu.set_gap_speed(0.5)
time.sleep(0.5)
print(ivu.gap_speed)

lifetime = epics.PV('SI-Glob:AP-CurrInfo:Lifetime-Mon')
inj_eff = epics.PV('SI-Glob:AP-CurrInfo:InjEff-Mon')

tune = Tune(Tune.DEVICES.SI)
tune.wait_for_connection(timeout=timeout)
print('Tune connected: ', tune.connected)

Tune connected:  True


In [7]:
sofb = SOFB(SOFB.DEVICES.SI)
fofb = HLFOFB(HLFOFB.DEVICES.SI)

### Make sure SOFB and FOFB loops are closed

In [ ]:
sofb.cmd_turn_on_autocorr()
fofb.cmd_turn_on_loop_state()

# Meas injection efficiency and lifetime

In [19]:
gaps = np.arange(24, 21, -1)
print(gaps)

[24 23 22]


In [29]:
nr_acq = 10
acq_sleep = 0.5
seconds_per_gap = 300

data = dict()
data['gap'] = list()
data['gap_mon'] = list()
data['timestamps'] = list()
data['gap_speed'] = list()
data['tunex'] = list()
data['tuney'] = list()
data['inj_eff'] = list()
data['lifetime'] = list()
data['beamline'] = IDSearch.conv_idname_2_beamline(ivu.devname)
for i, gap in enumerate(gaps):
    sucess = move_ivu_gap_robust(ivu, gap=gap, timeout=timeout, verbose=True)

    # Verify if undulator moved
    if not sucess:
        break
    
    gap_mon = ivu.gap_mon
    t0 = time.time()
    deltat = 0
    while deltat < seconds_per_gap:
    
        time.sleep(3)
        # Record info
        t = time.time()
        data['gap'].append(gap)
        data['gap_mon'].append(gap_mon)
        data['timestamps'].append(t)
        data['gap_speed'].append(ivu.gap_speed)
        data_acq = get_data(inj_eff, lifetime, tune, nr_acq, acq_sleep)
        tunex, tuney, inj_eff_data, lifetime_data = data_acq
        data['tunex'].append(tunex)
        data['tuney'].append(tuney)
        data['inj_eff'].append(inj_eff_data)
        data['lifetime'].append(lifetime_data)
        deltat = t - t0

    print(f'Done! ETA: {time.time()-t0:.3f}s')
    save(data, 'inj_eff_IVU_{}_gap_{:.1f}.pickle'.format(IDSearch.conv_idname_2_beamline(ivu.devname), gap))

24
Done! ETA: 40.393s
23
Done! ETA: 40.401s
22
Done! ETA: 40.429s
